
**This notebook produces the maximum layer slope figure based on a simplified SIA model.**

The change with depth of the vertical velocity may be written as (in a flow-aligned 2D coordinate system):

$\frac{\partial w}{\partial z} = \frac{\partial^2 l}{\partial t \partial z} + \frac{\partial u_f}{\partial z} \tan \alpha + u_f \sec^2(\alpha) \frac{\partial \alpha}{\partial z}$

Assuming the layers to be flat ($\alpha \approx 0$) is convenient because it eliminates both dependencies on $u_f$, the horizontal velocity in the along-flow direction.

This notebook explores the impact of the 2nd term: $\frac{\partial u_f}{\partial z} \tan \alpha$

In order to do this, we make some rough assumptions to provide a plausible upper end prediction of how large this term could be.

We assume:
* The ice velocity can be modelled by the shallow ice approximation (SIA) with some known value of n (n=3 in this example)
* The basal velocity is zero (unrealistic in fast flowing areas, but designed to provide an upper end prediction)
* We neglect local derivatives of flow speed and ice thickness: $\frac{\partial u_f}{\partial x}=\frac{\partial u_f}{\partial y}=\frac{\partial H}{\partial x}=\frac{\partial H}{\partial y}=0$

We use surface velocity and ice thickness measurements as inputs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import numpy as np
import dask

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs

import matplotlib.pyplot as plt
import cartopy.feature as cfeature

import scipy.ndimage

In [ ]:
# Load surface velocity and ice thickness datasets
# Re-interpolate all of them to a common grid spacing ()

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    surface_velocity = xr.open_dataset('greenland_stability_analysis/data/GRE_G0120_0000.nc', chunks={'x': 'auto', 'y': 'auto'}).coarsen(x=25, y=25, boundary='trim').mean()

grid_spacing_m = np.abs(np.median(np.diff(surface_velocity['x'])))
assert(np.abs(np.max(np.diff(surface_velocity['x']))) == grid_spacing_m)
assert(np.abs(np.min(np.diff(surface_velocity['x']))) == grid_spacing_m)
assert(np.abs(np.max(np.diff(surface_velocity['y']))) == grid_spacing_m)
assert(np.abs(np.min(np.diff(surface_velocity['y']))) == grid_spacing_m)

bedmachine = xr.open_dataset('greenland_stability_analysis/data/BedMachineGreenland-v5.nc')
H_inpt = bedmachine['thickness'].interp(x=surface_velocity['x'], y=surface_velocity['y'])

print(f"Grid spacing is {grid_spacing_m/1e3} km")

In [ ]:
# Quick helper for plotting on a map with a useful projection (EPSG:3413)

crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)
# In theory should work but doens't: crs_3413 = ccrs.epsg(3413)

def plot_on_map(da, **kwargs):
    if 'cmap' not in kwargs:
        if da.min() >= 0:
            kwargs['cmap'] = 'OrRd'
        else:
            kwargs['cmap'] = 'RdBu_r'
            if 'clim' not in kwargs:
                abs_max = float((np.abs(da)).max())
                kwargs['clim'] = (-abs_max, abs_max)

    plot = da.hvplot.quadmesh(x='x', y='y', aspect='equal', crs=crs_3413, **kwargs)
    plot = plot * gv.feature.coastline(projection=crs_3413)
    return plot

In [ ]:
# Smoothing the inputs helps to weird situations like rapidly changing converging and diverging surface flow that causes negative dw/dz values

smoothing_kernel_std = 5000 # m

us_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / grid_spacing_m, 'truncate': 4, 'mode': 'nearest'}, dask='allowed')
vs_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vy'], kwargs={'sigma': smoothing_kernel_std / grid_spacing_m, 'truncate': 4, 'mode': 'nearest'}, dask='allowed')
H_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, H_inpt, kwargs={'sigma': smoothing_kernel_std / grid_spacing_m, 'truncate': 4, 'mode': 'nearest'}, dask='allowed')

us = us_filt #surface_velocity['vx']
vs = vs_filt #surface_velocity['vy']
H = H_filt #H_inpt
# Filter out very thin ice -- anything less than 100 m isn't worth considering here
H.values[H.values < 100] = np.nan

In [ ]:
plot_on_map(H, frame_height=600, clim=(0, 4000), title="Ice thickness", clabel="Ice Thickness [m]")

In [ ]:
plot_on_map(us, frame_height=600, title="u (x velocity)", clabel="u [m/yr]", clim=(-50, 50))

In [ ]:
plot_on_map(vs, frame_height=600, title="v (y velocity)", clabel="v [m/yr]", clim=(-50, 50))

We assume an SIA model with constant temperature and zero basal sliding. Under this model:

$ \vec{v}(z) = \vec{v}(s) \left( 1 - \left( \frac{s-z}{H} \right)^{n+1} \right) $

In each direction (x and y), we can find the spatial derivatives:

$ \frac{\partial u(z)}{\partial x} = \frac{\partial u(s)}{\partial x} \left( 1 - \left( \frac{s-z}{H} \right)^{n+1} \right) -
    u(s) \left((n+1)\left(\frac{s-z}{H}\right)^n
    \left( \frac{1}{H} \frac{\partial s}{\partial x} - 
    \frac{s-z}{H^2} \frac{\partial H}{\partial x}\right) \right) $

Neglecting local changes in ice thickness and the bed, this simplifies to:

$
    \frac{\partial u(z)}{\partial x} \approx
    \frac{\partial u(s)}{\partial x} \left( 1 - \left( \frac{s-z}{H} \right)^{n+1} \right) = \frac{\partial u(s)}{\partial x} \left( 1 - \xi^{n+1} \right)
$

Where $\xi = \frac{s-z}{H}$.

Using incompressibility, we can find the vertical strain rate:

$
\frac{\partial w}{\partial z}
= -\left(1 - \xi^{n+1}\right) \nabla_H \cdot \vec{v} \left(s\right) $
$= - \left(1 - \xi^{n+1} \right) \left( \frac{\partial u_s}{\partial x} + \frac{\partial v_s}{\partial y} \right) $

In [ ]:
xi = 0.5 # (s-z)/H (fractional ice thickness)
n = 3 # Glen's flow law exponent

# Relevant derivatives

dus_dx = us.differentiate('x')
dvs_dy = vs.differentiate('y')

surface_velocity_divergence = dus_dx + dvs_dy
dw_dz = 1 * (1 - xi**(n+1)) * surface_velocity_divergence # TODO: investigate sign

du_dz = us * ((n+1)/H) * xi**n
dv_dz = vs * ((n+1)/H) * xi**n
dhorizontal_velocity_dz = np.sqrt(du_dz**2 + dv_dz**2)

In [ ]:
plot_on_map(surface_velocity_divergence, frame_height=600, title="dus/dx + dvs/dx", clabel="divergence", clim=(-1e-3, 1e-3))

In [ ]:
plot_on_map(dw_dz, frame_height=600, title="dw/dz", clabel="dw/dz", clim=(-1e-3, 1e-3))

In [ ]:
dw_dz.plot.hist(bins=500, figsize=(10, 5), range=(-1e-2, 1e-2))

In [ ]:
dw_dz_values = dw_dz.to_numpy().flatten()
dw_dz_values = dw_dz_values[~np.isnan(dw_dz_values)]
np.mean(dw_dz_values >= 0), np.mean(dw_dz_values < 0)

In [ ]:
max_error_pct = 0.1
# Constrain dw/dz to be positive -- doesn't make sense for it to be negative under this simple SIA model
max_layer_slope = np.abs(np.arctan(max_error_pct * np.maximum(dw_dz, 0) / dhorizontal_velocity_dz))

In [ ]:
p = plot_on_map(max_layer_slope * (180/np.pi), cmap='cividis', clim=(0, 2), clabel="Slope [degrees]", frame_height=800, title=f"Maximum layer slope (deg) for 10% error in vertical strain\nrate when neglecting layer slope\nat {xi*100}% fractional ice thickness")

p

In [ ]:
# Save p to a PNG
import holoviews as hv
hv.extension('bokeh')
hv.save(p, 'max_layer_slope.png', fmt='png')